### The goal is to approximate a 2d interpolation funciton by using 1d interpolation funcitons

In [146]:
import sys
sys.path.insert(0,'..')

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from algorithms import create_interp_1d_funcs, create_interp_2d_funcs, sigma_f1d, sigma_f2d
from data_utils import create_1d_data, create_2d_data
%matplotlib notebook


## Test approximate vs. 2d interpolation

In [231]:
        def f(x):
            return np.array([datum[0]*np.sin(datum[1]) + datum[1] + datum[2] for datum in x]).reshape(-1, 1)

        # ----creating training data----
        primary_cut_center = [0, 0, 0]
        x_1d = create_1d_data(x_range=(-8, 8, 1), cut_center=primary_cut_center)
        x_2d = create_2d_data(x1_range=(-8, 8, 1), x2_range=(-8, 8, 1), cut_center=primary_cut_center)
        y_1d = f(x_1d)
        y_2d = f(x_2d)

        # ----creating the interpolation functions----
        f0 = f(np.array([primary_cut_center]))
        f_1d = create_interp_1d_funcs(x_1d, y_1d, primary_cut_center)
        f_2d = create_interp_2d_funcs(x_2d, y_2d, primary_cut_center)

        # ----Testing----
        test_datum = [1.45, 2.2823, 0.2782]

In [232]:
# ---- the negative part of sigma2d for f01-----
neg_part = -(f_1d[0](test_datum[0]) - f0) - (f_1d[1](test_datum[1]) - f0) - f0
print(f'net_part of sigma_2d for f01: {neg_part}')

net_part of sigma_2d for f01: [[-2.2823]]


In [233]:
# ----testing the first 2d interpolated funciton --> f01----

f01_interpolated = f_2d[0][0](test_datum[0], test_datum[1])
print(f'f01_interpolated: {f01_interpolated}')

f01_interpolated: [3.37773063]


In [236]:
# ----approximating f01----
secondary_cut_center = [1, 1 ,primary_cut_center[2]] # or different cutcenter altoghether? TESTED: Z has to be the same as primary cut_center
secondary_x_1d = create_1d_data(x_range=(-8, 8, 1), cut_center=secondary_cut_center)
secondary_y_1d = f(secondary_x_1d)
secondary_f_1d = create_interp_1d_funcs(secondary_x_1d, secondary_y_1d, secondary_cut_center)
secondary_f0 = f(np.array([secondary_cut_center]))

f01_approximated = secondary_f0 + (secondary_f_1d[0](test_datum[0]) - secondary_f0) + (secondary_f_1d[1](test_datum[1]) - secondary_f0) 
print(f'f01_approximated: {f01_approximated}')

f01_approximated: [[3.42484694]]


Surprizingly changing the **x and y** coordinate of the **secondary_cut_center** does **NOT** change the **f01_approximated**. This is the same as saying:

$$F_{01}(x, y, \hat{z}) = F(\hat{x}, \hat{y}, \hat{z}) + [F(x, \hat{y}, \hat{z}) - F(\hat{x}, \hat{y}, \hat{z})] +  [F(\hat{x}, y, \hat{z}) - F(\hat{x}, \hat{y}, \hat{z})]$$ 
is the same as:
$$F_{01}(x, y, \hat{z}) = F(\bar{x}, \bar{y}, \hat{z}) + [F(x, \bar{y}, \hat{z}) - F(\bar{x}, \bar{y}, \hat{z})] +  [F(\bar{x}, y, \hat{z}) - F(\bar{x}, \bar{y}, \hat{z})]$$

where primary cut center: $$(\hat{x}, \hat{y}, \hat{z})$$ 
and secondary cut center: $$(\bar{x}, \bar{y}, \hat{z})$$

**Turns out this is only the case if the x, and y variables are decouples (are linear) in the F function**